# Change of sampling rate

The sampling rate of a station may change over the years. 
If that happens the mseed-file in the
SDS contains multiple traces with different sampling rates. 
These can not be merged without
resampling to a common rate. We choose to resample the 
stream to the highest rate. 
This happens in the routing `util.process_streams()` which calls 
`util.merge_different_samplingrates`.

In [ ]:
from importlib import reload
import os

import numpy as np
from obspy.core import UTCDateTime as UTC, read, Stream

from data_quality_control import sds_db, dqclogging, util

Set parameters for a station and time with known change in SR.

In [ ]:
network = 'GR'
station = 'BFO'
location = ''
channel = 'HHZ'

overlap = 60 #3600
fmin, fmax = (4, 14)
nperseg = 2048
winlen_in_s = 3600
proclen = 24*3600

outdir = 'output/'

sds_root = os.path.abspath('/sds')
inventory_routing_type = "eida-routing"


startdate = UTC("2011-10-17")
enddate = UTC("2011-10-20")

# Set verbosity: "ERROR" < "WARNING" < "INFO" < "DEBUG"
#sds_db.logger.setLevel("INFO")
#sds_db.base.logger.setLevel("INFO")
dqclogging.configure_handlers(sds_db.logger, "DEBUG", "DEBUG", "dqc_processing_test.log")

Run implemented processing. Note the line

    data_quality_control.util - INFO - Found different sampling rates. Resampling to highest (100 Hz).
    
in the log messages indicating a change in SR.

In [ ]:
reload(sds_db)
reload(util)
processor = sds_db.SDSProcessor(
                network, station, channel, 
                inventory_routing_type, 
                sds_root, outdir=outdir, 
                overlap=overlap, nperseg=nperseg, 
                winlen_seconds=winlen_in_s, 
                proclen_seconds=proclen,
                amplitude_frequencies=(fmin, fmax))

print(processor)

try:
        processor.process(startdate, enddate, force_new_file=True)
except Exception as e:
        processor.logger.error(str(e))

To verify, we load the data of this day and find that there are
1 trace with SR = 80 Hz and 2 with SR = 100 Hz.

In [ ]:
time = UTC("2022-10-19")
st = read("/sds/2011/GR/BFO/HHZ.D/GR.BFO..HHZ.D.2011.{:d}".format(time.julday))
print(st)

The function in which this problem occurs looks like this:

```python
    # %load -n util.merge_different_samplingrates
    def merge_different_samplingrates(st):
        """
        Merge stream, fill gaps with nans.
        If traces have different sampling rates, resample to highest.
        """
        try:
            st.merge(fill_value=np.nan)
        except Exception:
            sr = max([tr.stats.sampling_rate for tr in st])
            module_logger.info("Found different sampling rates. " + 
                        "Resampling to highest ({:g} Hz).".format(sr))
            st.resample(sr)
            st.merge(fill_value=np.nan)
```

First, let's view the Exception:

In [ ]:
st = read("/sds/2011/GR/BFO/HHZ.D/GR.BFO..HHZ.D.2011.{:d}".format(time.julday))
print(st)

st.merge(fill_value=np.nan)

So obspy complains that the ids are the same but traces have different sampling rates.
If we resample to a common rate, as done in the `except` part of the routine, 
we can merge.

In [ ]:
sr = max([tr.stats.sampling_rate for tr in st])
st.resample(sr)
st.merge(fill_value=np.nan)

print(st)